# Notebook destiné à faire du résumé en découpant plusieurs fois un long texte

## Import du modèle déjà fine tuned

In [1]:
from llama_cpp import Llama

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = Llama(
  model_path="mistral-7b-instruct-v0.2.Q4_K_M.gguf",  # Download the model file first
  n_ctx=32768,  # The max sequence length to use - note that longer sequence lengths require much more resources
  n_threads=4,            # The number of CPU threads to use, tailor to your system and the resulting performance
  n_gpu_layers=0         # The number of layers to offload to GPU, if you have GPU acceleration available
)


llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 l

### Définition de l'output attendu

In [2]:
# Simple inference example
output = llm(
  "<s>[INST] {prompt} [/INST]", # Prompt
  max_tokens=512,  # Generate up to 512 tokens
  stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
  echo=True        # Whether to echo the prompt
)



llama_print_timings:        load time =   42156.08 ms
llama_print_timings:      sample time =      65.33 ms /   193 runs   (    0.34 ms per token,  2954.46 tokens per second)
llama_print_timings: prompt eval time =   42154.28 ms /    13 tokens ( 3242.64 ms per token,     0.31 tokens per second)
llama_print_timings:        eval time =   76242.51 ms /   192 runs   (  397.10 ms per token,     2.52 tokens per second)
llama_print_timings:       total time =  119322.77 ms /   205 tokens


### Test sur une requete

In [28]:
# Chat Completion API

llm = Llama(model_path="./mistral-7b-instruct-v0.2.Q4_K_M.gguf", chat_format="llama-2")  # Set chat_format according to the model you are using
summary = llm.create_chat_completion(
    messages = [
        {"role": "system", "content": "You job is to make summaries of long texts."},
        {
            "role": "user",
            "content": "summarize harry potter 1"
        }
    ]
)



llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from ./mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                

#### Sortons uniquement l'ouput de la requete qui nous intéresse

In [40]:
liste_dictionnaires = summary["choices"][0]['message']['content']

print(liste_dictionnaires)

 Title: Harry Potter and the Philosopher's Stone (Sorcerer's Stone in some editions)

Summary:

Harry Potter, an orphan boy living with his abusive aunt and uncle, receives a letter on his eleventh birthday inviting him to attend Hogwarts School of Witchcraft and Wizardry. Unbeknownst to Harry, he is a famous survivor of Voldemort, a dark wizard who had tried to kill him as an infant. At Hogwarts, Harry learns about his magical abilities from his kind mentor, Professor Albus Dumbledore, and makes new friends, Hermione Granger and Ron Weasley.

The trio hears rumors of a legendary Philosopher's (Sorcerer's) Stone that can turn any metal into gold and produce the Elixir of Life, granting immortality. Harry learns that Voldemort is after this stone to regain his youth and power. With Dumbledore's guidance, they embark on a quest to protect the stone from falling into Voldemort's hands.

The trio faces numerous challenges, including battling trolls, cheating in exams, and dealing with a mi

## Fonctions

In [48]:
# Diviser le texte en segments de 500 caractères ou moins
def split_text_into_segments(text, segment_length=1000, instruction="make me a short summary of the following text:"):
    segments = []
    current_segment = ""

    words = text.split()
    for word in words:
        # Ajouter le mot au segment en cours
        current_segment += word + " "

        # Vérifier si le segment en cours dépasse la longueur maximale
        if len(current_segment) >= segment_length:
            # Ajouter l'instruction au début du segment
            segment_with_instruction = [f"{instruction} {current_segment.strip()}"]
            # Ajouter le segment complet à la liste des segments
            segments.append(segment_with_instruction)
            # Réinitialiser le segment en cours
            current_segment = ""

    # Ajouter le segment final s'il n'a pas été ajouté
    if current_segment:
        # Ajouter l'instruction au début du segment final
        segment_with_instruction = [f"{instruction} {current_segment.strip()}"]
        segments.append(segment_with_instruction)

    return segments


In [41]:
# Résumez chaque segment
def summarize_text_segments(segments):
    summaries = []
    for segment in segments:
        print("start of segment")
        requete = llm.create_chat_completion(
                messages = [
                    {"role": "system", "content": "You job is to make summaries of long texts."},
                    {
                        "role": "user",
                        "content": f"{segment}"
                    }
                    ]
                )
        
        summary = [requete["choices"][0]['message']['content']]
        summaries.append(summary)
    return summaries


In [59]:
# Rassemblez les résumés et résumez-les à nouveau
def summarize_summaries(summaries):
    concatenated_summaries = ""
    for summary in summaries:
        concatenated_summaries += str(summary) + " "
    final_summary = llm.create_chat_completion(
                messages = [
                    {"role": "system", "content": "You job is to make summaries of long texts."},
                    {
                        "role": "user",
                        "content": f"Make me a short summary of this : {concatenated_summaries}"
                    }
                    ]
                )
    return final_summary['choices'][0]['message']['content']

## Applications à un long texte extrait de livre

### Import d'un dataset contenant des chapitres de livres

In [42]:
from datasets import load_dataset
dataset = load_dataset("pszemraj/booksum-short")

In [43]:
# Votre très long texte
long_text = dataset["train"]["chapter"][0]

#### division du long texte en segments

In [49]:
# Divisez le texte en segments
text_segments = split_text_into_segments(long_text)

In [50]:
print(text_segments[0])

['make me a short summary of the following text: "Before these fields were shorn and tilled, Full to the brim our rivers flowed; The melody of waters filled The fresh and boundless wood; And torrents dashed, and rivulets played, And fountains spouted in the shade." BRYANT. Leaving the unsuspecting Heyward and his confiding companions to penetrate still deeper into a forest that contained such treacherous inmates, we must use an author\'s privilege, and shift the scene a few miles to the westward of the place where we have last seen them. On that day, two men were lingering on the banks of a small but rapid stream, within an hour\'s journey of the encampment of Webb, like those who awaited the appearance of an absent person, or the approach of some expected event. The vast canopy of woods spread itself to the margin of the river overhanging the water, and shadowing its dark current with a deeper hue. The rays of the sun were beginning to grow less fierce, and the intense heat of the day

### Résumons les différentes parties de ce chapitre

In [51]:
# Résumez chaque segment
segment_summaries = summarize_text_segments(text_segments)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   35272.35 ms
llama_print_timings:      sample time =      42.58 ms /    84 runs   (    0.51 ms per token,  1972.62 tokens per second)
llama_print_timings: prompt eval time =   74919.71 ms /   260 tokens (  288.15 ms per token,     3.47 tokens per second)
llama_print_timings:        eval time =   63675.06 ms /    83 runs   (  767.17 ms per token,     1.30 tokens per second)
llama_print_timings:       total time =  139150.16 ms /   343 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   35272.35 ms
llama_print_timings:      sample time =      51.25 ms /   124 runs   (    0.41 ms per token,  2419.37 tokens per second)
llama_print_timings: prompt eval time =   65581.91 ms /   243 tokens (  269.88 ms per token,     3.71 tokens per second)
llama_print_timings:        eval time =   60305.41 ms /   123 runs   (  490.29 ms per token,     2.04 tokens per second)
llama_print_timings:       to

In [52]:
print(segment_summaries)

[[" In this text, Bryant describes how before being cultivated, rivers flowed abundantly through the woods. Then, the scene shifts to two men waiting by a small rapid stream, near Webb's encampment. They are anticipating someone or an event. The sun's rays are becoming less intense and the heat is decreasing as cooler vapors rise from the springs."], [' In a quiet, sultry American landscape in July, two foresters sat near a secluded spot with fountains rising from the foliage. The peaceful scene was interrupted only by the sounds of woodpeckers, jays, and a distant waterfall. Despite these distractions, their attention was focused on their intense dialogue. One forester had native features and wore traditional woodland attire, while the other had sunburnt European descent, hinted through his rugged appearance. The former sat on a mossy log, using expressive gestures to emphasize his words.'], [' A Native American warrior stood before the crowd, nearly naked with a body adorned in contr

## Rajoutons une fonction pour regroupe les résumés entre eux, et refaire du résumé de résumé

In [76]:
def group_summaries(summaries, group_size=3):
    grouped_summaries = []
    current_group = []

    for summary_list in summaries:
        # Convertir la liste de résumés en une seule chaîne de caractères
        summary_string = ' '.join(summary_list)
        current_group.append(summary_string)
        
        if len(current_group) == group_size:
            grouped_summaries.append(current_group)
            current_group = []

    # Ajouter le dernier groupe s'il est incomplet
    if current_group:
        grouped_summaries.append(current_group)

    return grouped_summaries

In [77]:
summary_step_2 = group_summaries(segment_summaries)

In [84]:
print(summary_step_2[6])

[" The Native American, standing up with dignity, addressed Hawkeye, telling him to speak to his brothers. Hawkeye agreed and promised to use English so the king wouldn't be embarrassed. Despite not seeing or hearing anyone, Hawkeye believed he heard sounds of people approaching and warned them to stay away from the Iroquois."]


In [85]:
# Résumez chaque segment
segment_summaries_2 = summarize_text_segments(summary_step_2)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   35272.35 ms
llama_print_timings:      sample time =      63.88 ms /   166 runs   (    0.38 ms per token,  2598.74 tokens per second)
llama_print_timings: prompt eval time =   80423.34 ms /   321 tokens (  250.54 ms per token,     3.99 tokens per second)
llama_print_timings:        eval time =   71193.77 ms /   165 runs   (  431.48 ms per token,     2.32 tokens per second)
llama_print_timings:       total time =  152788.29 ms /   486 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   35272.35 ms
llama_print_timings:      sample time =      18.57 ms /    61 runs   (    0.30 ms per token,  3284.51 tokens per second)
llama_print_timings: prompt eval time =  117432.04 ms /   425 tokens (  276.31 ms per token,     3.62 tokens per second)
llama_print_timings:        eval time =   24899.38 ms /    60 runs   (  414.99 ms per token,     2.41 tokens per second)
llama_print_timings:       to

In [86]:
print(segment_summaries_2)

[[" The text describes two scenes in an American landscape in July. In the first scene, rivers flowed abundantly through the woods before being cultivated. Two men are waiting near a small rapid stream, anticipating someone or an event. The sun's heat is decreasing as cooler vapors rise from the springs.\n\nIn the second scene, two foresters sit in a quiet, sultry landscape with fountains rising from the foliage. They are engaged in an intense dialogue, interrupted only by the sounds of woodpeckers, jays, and a distant waterfall. One forester has native features and wears traditional woodland attire, while the other has sunburnt European descent.\n\nIn another scene, a Native American warrior stands before a crowd, nearly naked"], [' The text describes a hunter with a lean, muscular build and keen eyes. He wears traditional hunting gear and carries a long rifle. Hawk-eye, the hunter, speaks to Chingachgook in their native language. They discuss their shared histories of conflict over l

#### notre résumé est toujours trop long par rapport à nos max tokens du débuts, réduisons encore

In [93]:
# Résumez chaque segment
summaries_step_3 = group_summaries(segment_summaries_2)

In [92]:
print(summaries_step_3[2])

[" A Native American speaker addressed Hawkeye, asking him to communicate with his brothers. Hawkeye agreed and promised to use English to avoid embarrassing the king. Although he didn't see or hear anyone, Hawkeye believed he heard approaching sounds and warned the Iroquois to keep their distance."]


In [94]:
# Résumez chaque segment
segment_summaries_3 = summarize_text_segments(summaries_step_3)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   35272.35 ms
llama_print_timings:      sample time =      32.66 ms /    46 runs   (    0.71 ms per token,  1408.36 tokens per second)
llama_print_timings: prompt eval time =  107146.50 ms /   439 tokens (  244.07 ms per token,     4.10 tokens per second)
llama_print_timings:        eval time =  281113.72 ms /    45 runs   ( 6246.97 ms per token,     0.16 tokens per second)
llama_print_timings:       total time =  388823.64 ms /   484 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   35272.35 ms
llama_print_timings:      sample time =       3.58 ms /     9 runs   (    0.40 ms per token,  2511.16 tokens per second)
llama_print_timings: prompt eval time =  113840.93 ms /   477 tokens (  238.66 ms per token,     4.19 tokens per second)
llama_print_timings:        eval time =    4608.02 ms /     8 runs   (  576.00 ms per token,     1.74 tokens per second)
llama_print_timings:       to

In [95]:
# Résumez les résumés
final_summary = summarize_summaries(segment_summaries_3)

print(final_summary)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   35272.35 ms
llama_print_timings:      sample time =      46.18 ms /    90 runs   (    0.51 ms per token,  1948.90 tokens per second)
llama_print_timings: prompt eval time =   47982.18 ms /   139 tokens (  345.20 ms per token,     2.90 tokens per second)
llama_print_timings:        eval time =   42846.28 ms /    89 runs   (  481.42 ms per token,     2.08 tokens per second)
llama_print_timings:       total time =   91775.57 ms /   228 tokens


 In the first part of the text, two scenes in an American landscape in July are described: men waiting by a small river as cool vapors rise from springs due to decreasing heat. In the second part, a Native American speaker asks Hawkeye to deliver a message to his brothers using English to avoid embarrassment for their chief. Hawkeye agrees and senses approaching sounds, urging caution for the Iroquois.


In [96]:
print(long_text)


  "Before these fields were shorn and tilled,
    Full to the brim our rivers flowed;
  The melody of waters filled
    The fresh and boundless wood;
  And torrents dashed, and rivulets played,
    And fountains spouted in the shade."

  BRYANT.


Leaving the unsuspecting Heyward and his confiding companions to
penetrate still deeper into a forest that contained such treacherous
inmates, we must use an author's privilege, and shift the scene a few
miles to the westward of the place where we have last seen them.

On that day, two men were lingering on the banks of a small but rapid
stream, within an hour's journey of the encampment of Webb, like those
who awaited the appearance of an absent person, or the approach of some
expected event. The vast canopy of woods spread itself to the margin of
the river overhanging the water, and shadowing its dark current with a
deeper hue. The rays of the sun were beginning to grow less fierce, and
the intense heat of the day was lessened, as the cool